<a href="https://colab.research.google.com/github/HaqTetsuya/rusdi-prototype-1/blob/main/test_fuzzy_string.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rapidfuzz

In [ ]:
from rapidfuzz import fuzz

# Step 1: Entities Dictionary
entities = {
    "wifi": ["wifi", "internet"],
    "ruang_baca": ["ruang baca", "reading room"],
    "ruang_diskusi": ["ruang diskusi", "discussion room"],
    "komputer": ["komputer", "PC umum"],
    "kantin": ["kantin", "cafeteria"],
    "toilet": ["toilet", "kamar mandi"],
    "mushola": ["mushola", "tempat sholat"],
    "parkir": ["parkiran", "tempat parkir"],
    "printer": ["printer", "fotokopi", "scanner"],
}

# Step 2: Updated Fuzzy Extraction (Multiple Entities)
def extract_entities_fuzzy(user_text, entities, threshold=80):
    user_text = user_text.lower()
    detected_entities = set()

    for entity_name, keywords in entities.items():
        for keyword in keywords:
            score = fuzz.partial_ratio(keyword, user_text)
            if score >= threshold:
                detected_entities.add(entity_name)
                break  # satu keyword match sudah cukup untuk satu entity

    return list(detected_entities)

# Step 3: Handler (return array of entities)
def handle_facility_intent(user_text):
    detected_entities = extract_entities_fuzzy(user_text, entities)
    return detected_entities

# Step 4: Testing
test_messages = [
    "Apakah ada wiffi gratis?",
    "Saya mau ke ruang diskus dan ruang baca",
    "Apakah ada komputer buat umum dan toilet di sini?",
    "Dimana saya bisa fotocopy dan print?",
    "Ada cafetaria dan mushola?",
    "Saya ingin tahu tentang parkir dan internet",
]

for msg in test_messages:
    print(f"User: {msg}")
    print(f"Detected Entities: {handle_facility_intent(msg)}\n")
